### Introduction to environment 
This time we will teach our self driving car to drive us home (orange node). We have to be careful though as some streets are under construction (grey node) and we don’t want our car crashing into it.<br>
<br>
As you can see we have streets numbered from 0 to 8. This gives us 9 unique **states** (streets). At any given time, our car (agent) can be in one of this 9 states. State 8 is our Goal, also called as a **terminal state**.<br>
<br>
Our car can go left, right, up and down. To put it differently, our agent can take four different **actions**. We write it as: a ∈ A{up,down,left,right}<br>
<br>
The agent receives a **reward 10 for reaching the terminal state**, other than that there are no rewards.
![GirdWorld](http://i.imgur.com/C1fj5ZE.png)

In [1]:
import gym
import sys
import itertools
import numpy as np
#import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
# Create the environment
env_list = ['FrozenLake-v0','Taxi-v2']
env = gym.make(env_list[0])


[2017-04-04 10:23:33,716] Making new env: FrozenLake-v0


### Q-learning and Q-table 

Now we’ll create a matrix, “Q” also called as Q-table, this will be the brain of our agent. The matrix Q is initialized to zero, as agent starts out knowing nothing (just like John Snow;)). It updates Q-table with new values for state-action pair, as it learns. Here is the formula to calculate the Q[state, action] <br> 
<br>
Q[s, a] = Q[s, a] + alpha*(R + gamma*Max[Q(s’, A)] - Q[s, a]) <br>
<br>
Where;<br>
alpha is the **learning rate**,<br>
gamma is the **discount factor**. It quantifies how much importance we give for future rewards. It’s also handy to approximate the noise in future rewards. Gamma varies from 0 to 1. If Gamma is closer to zero, the agent will tend to consider only immediate rewards. If Gamma is closer to one, the agent will consider future rewards with greater weight, willing to delay the reward.<br>
Max[Q(s’, A)] gives **a maximum value of Q for all possible actions in the next state**.<br>
<br>
The Agent explores different ‘state-action’ combinations till it reaches the goal or falls into the hole. We will call each of this explorations an **episode**. Each time the agent arrives at goal or is terminated, we start with next episode.

### Some easy maths to demo how Q-table is updated:
![Q-table](http://i.imgur.com/WI0Dele.png)
take learning rate (alpha) as 0.5 & discount factor (gamma) as 0.9 <br>
Q[s, a] = Q[s, a] + alpha*(R + gamma*Max[Q(s’, A)] — Q[s, a])<br>
Early Episodes<br>
Q[3,L] = Q[3,L]+0.5*(10+0.9*Max[Q(8,U),Q(8,D),Q(8,R),Q(8,L)]-Q(3,L)) <br>
Q[3,L] = 0 + 0.5 * (10 + 0.9 * Max [0, 0, 0, 0] -0)<br>
Q[3,L] = 5, Similarly Q[6,D] = 5<br>
Next Episodes<br>
Q[2,L] = Q[2,L]+0.5*(0+0.9*Max[Q(6,U),Q(6,D),Q(6,R),Q(6,L)]-Q(2,L))<br>
Q[2,L] = 0 + 0.5 * (0 + 0.9 * Max [0, 5, 0, 0] -0)<br>
Q[2,L] = 2.25, Similarly Q[2,D] = 2.25 and Q[7,L] = 2.25<br>
Eventually<br>
Q[1,D] = 1.0125 and Q[0,L] = 0.455625<br>

### Exploration and Exploitation — Epsilon (ε)
As agent begins the learning, we would want it to take random actions to explore more paths. But as the agent gets better, the Q-function converges to more consistent Q-values. Now we would like our agent to exploit paths with highest Q-value i.e takes greedy actions. This is where epsilon comes in.<br> 
<br> 
The agent takes random actions for probability ε and greedy action for probability (1-ε). <br> 
<br> 
Google DeepMind used a decaying ε-greedy action selection. Where ε decays over time from 1 to 0.1 — in the beginning, the system makes completely random moves to explore the state space maximally, and then it settles down to a fixed exploration rate.

### Pseudo Code
![Q-learning](http://i.imgur.com/EeZcNeR.png)

In [3]:
def q_learning(env, num_episodes, alpha=0.85, discount_factor=0.99):
    """
    Q learning algorithm, off-polics TD control. Finds optimal gready policies
    Args:
    env: Given environment to solve
    num_episodes: Number of episodes to learn
    alpha: learning rate
    discount factor: weight/importance given to future rewards
    epsilon: probability of taking random action. 
             We are using decaying epsilon, 
             i.e high randomness at beginning and low towards end
    Returns:
    Optimal Q
    """
     
    # decaying epsilon, i.e we will divide num of episodes passed
    epsilon = 1.0
    # create a numpy array filled with zeros 
    # rows = number of observations & cols = possible actions
    Q = np.zeros([env.observation_space.n, env.action_space.n]) 
    
    for i_episode in range(num_episodes):
            # reset the env
            state = env.reset()
            # itertools.count() has similar to 'while True:'
            for t in itertools.count():
                # generate a random num between 0 and 1 e.g. 0.35, 0.73 etc..
                # if the generated num is smaller than epsilon, we follow exploration policy 
                if np.random.random() <= epsilon:
                    # select a random action from set of all actions
                    action = env.action_space.sample()
                # if the generated num is greater than epsilon, we follow exploitation policy
                else:
                    # select an action with highest value for current state
                    action = np.argmax(Q[state, :])
                
                # apply selected action, collect values for next_state and reward
                next_state, reward, done, _ = env.step(action)
                
                # Calculate the Q-learning target value
                Q_target = reward + discount_factor*np.max(Q[next_state,:])
                # Calculate the difference/error between target and current Q
                Q_delta = Q_target - Q[state,action]
                # Update the Q table, alpha is the learning rate
                Q[state, action] = Q[state, action] + (alpha * Q_delta)
                
                # break if done, i.e. if end of this episode
                if done:
                    break
                # make the next_state into current state as we go for next iteration
                state = next_state
            # gradualy decay the epsilon
            if epsilon > 0.1:
                epsilon -= 1.0/num_episodes
    
    return Q    # return optimal Q
    
    

In [4]:
def test_algorithm(env, Q):
    """
    Test script for Q function
    Args:
    env: Given environment to test Q function
    Q: Q function to verified
    Returns:
    Total rewards for one episode
    """
    
    state = env.reset()
    total_reward = 0
    
    while True:
        # selection the action with highest values i.e. best action
        action = np.argmax(Q[state, :])
        # apply selected action
        next_state, reward, done, _ = env.step(action)
        # render environment
        env.render()
        # calculate total reward
        total_reward += reward
        
        if done:
            print(total_reward)
            break
            
        state = next_state
    
    return total_reward 

In [11]:
Q = q_learning(env, 2000)

In [12]:
print(Q)

[[  5.81601034e-01   5.48947196e-01   5.71618900e-01   5.80961970e-01]
 [  7.73473571e-02   2.74745056e-02   5.20363800e-01   6.90664599e-01]
 [  2.27252795e-01   4.96515417e-01   1.37152480e-01   3.31289754e-01]
 [  4.73595482e-02   3.74819510e-02   3.51660510e-02   4.73937444e-01]
 [  5.96029365e-01   9.61543241e-02   5.49391393e-05   1.20234534e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.13649665e-02   1.68729101e-02   2.95534048e-04   3.34253584e-05]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.17522777e-01   1.80769975e-02   6.30194144e-01   7.58542005e-01]
 [  2.78574817e-03   8.55462125e-01   1.05827056e-01   3.04137863e-03]
 [  8.51346298e-01   1.39887216e-01   1.15799949e-02   1.10476723e-02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.43397267e-01   1.40706256e-01   9.39220229e-01   5.50304369e-01]
 [  8.

In [19]:
test_algorithm(env, Q)

  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
1.0


1.0

### Further resources
[Reinforcement Learning: An Introduction](http://people.inf.elte.hu/lorincz/Files/RL_2006/SuttonBook.pdf) — Chapter 6: Temporal-Difference Learning <br>
[David Silver’s RL Course Lecture 4](https://www.youtube.com/watch?v=PnHCvfgC_ZA) — Model-Free Prediction <br>
[David Silver’s RL Course Lecture 5](https://www.youtube.com/watch?v=0g4j2k_Ggc4) — Model-Free Control <br>

I hope this tutorial has been helpful to those new to Q-learning and TD-reinforcement learning!<br>
If you’d like to follow my writing on Reinforcement Learning, follow me on Medium [@Shreyas Gite](https://medium.com/@shreyas.gite), or on twitter [@shreyasgite](https://twitter.com/shreyasgite).<br>
Feel free to write to me for any questions or suggestions :)<br>